In [ ]:
%pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib openai

### LLM Agent 기반 자동 예약 서비스 구성하기

0. Anaconda 설치 및 로컬에서 python 환경 구축
해당 블로그 문서 [링크](https://kevinitcoding.tistory.com/entry/%EC%95%84%EB%82%98%EC%BD%98%EB%8B%A4%EB%9E%80-%EC%84%A4%EC%B9%98-%EB%B0%A9%EB%B2%95%EA%B9%8C%EC%A7%80-%ED%95%9C-%EB%B0%A9%EC%97%90-%EC%A0%95%EB%A6%AC#google_vignette)를 참고해서 설치해주세요. 아나콘다 설치는 구글 검색을 통해 많은 문서의 도움을 받으실 수 있습니다.

##### 1. 구글 캘린더 API 사용하기

1-1 Google Calendar 문서 [[링크]](https://developers.google.com/calendar/api/quickstart/python?hl=ko) 를 참고해서 OAuth 인증정보를 담은 파일을 다운로드 받아야 합니다. client를 생성하고 json 파일을 다운받습니다

1-2. [**해당 단계는 Colab이 아닌 본인의데스크탑 로컬 환경에서 진행해주세요**]
위에서 생성한 `client_secret_xxx.json` 파일을 `credentials.json`라는 이름으로 변경해줍니다. 그리고 강의자료에 `calendar_quickstart.py`와 같은 폴더에 위치시킵니다.

이후 python이 설치된 환경에서 ``python calendar_quickstart.py``를 실행시키면 `token.json` 이라는 인증 토큰이 만들어집니다.

1-3 구글 캘린더 api 사용하기 (calendar id를 가져와야 함, 구글 캘린더 => 설정 및 공유 에 들어가면 캘린더 id를 얻을 수 있음)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

dir_token = '/content/drive/MyDrive/데이터/'

In [ ]:
google_calendar_api_key = # Google Calendar API Key 입력
calendar_id = # email 주소 혹은 구글 캘린더의 id 입력

In [ ]:
from __future__ import print_function

import datetime
import os.path
import json
# import config
import requests

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

class GoogleCalendar:
    def __init__(self, calendar_id=None, google_calendar_api_key=None):
        self.SCOPES = ['https://www.googleapis.com/auth/calendar']
        self.file_path = os.path.join(dir_token, "token.json")
        self.calendar_id = calendar_id
        self.google_calendar_api_key = google_calendar_api_key

    def renew_google_token(self):
        creds = None
        if os.path.exists(self.file_path):
            creds = Credentials.from_authorized_user_file(self.file_path, self.SCOPES)
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    os.path.join(dir_token, 'credentials.json'), self.SCOPES)
                creds = flow.run_local_server(port=0)
            with open(self.file_path, 'w') as token:
                token.write(creds.to_json())
        return creds

    def get_google_token(self):
        self.renew_google_token()

        with open(self.file_path, 'r') as file:
            data = json.load(file)

        return data["token"]

    def get_today_google_calendar(self):
        calendar_id = self.calendar_id
        url = f"https://www.googleapis.com/calendar/v3/calendars/{self.calendar_id}/events"

        now = datetime.datetime.now(datetime.timezone.utc)
        now2 = now + datetime.timedelta(hours=1)

        time_min = now.isoformat()
        time_max = now2.isoformat()

        params = {"key": self.google_calendar_api_key, "timeMax": time_max, "timeMin": time_min}
        response = requests.get(url, params=params, headers={"Authorization": f"Bearer {self.get_google_token()}"})

        return response.json()

    def set_google_calendar(self, event):
        creds = self.renew_google_token()
        service = build('calendar', 'v3', credentials=creds)

        # calendarId : 캘린더 ID. primary이 기본 값
        service.events().insert(calendarId=self.calendar_id, body=event).execute()

    def delete_google_calendar(self, event_id):
        creds = self.renew_google_token()
        service = build('calendar', 'v3', credentials=creds)

        service.events().delete(calendarId=self.calendar_id, eventId=event_id).execute()

    def get_google_calendar(self, event_id):
        creds = self.renew_google_token()
        service = build('calendar', 'v3', credentials=creds)

        event = service.events().get(calendarId=self.calendar_id, eventId=event_id).execute()

        return event

    def get_event_id_list(self, num_events=10):
        creds = self.renew_google_token()
        service = build('calendar', 'v3', credentials=creds)

        events = service.events().list(calendarId=self.calendar_id).execute()
        event_id_list = [event["id"] for event in events["items"]]

        return event_id_list[-num_events:]

In [ ]:
google_calendar = GoogleCalendar(calendar_id = calendar_id, google_calendar_api_key = google_calendar_api_key)

In [ ]:
google_calendar.get_today_google_calendar()

In [ ]:
event_list= google_calendar.get_event_id_list()
event_list

In [ ]:
google_calendar.get_google_calendar(event_list[3])

In [ ]:
event = {
        'summary': '일정 제목', # 일정 제목
        'location': '서울특별시', # 일정 장소
        'description': '일정 설명', # 일정 설명
        'start': { # 시작 날짜
            'dateTime': "2024-04-19" + 'T09:00:00',
            'timeZone': 'Asia/Seoul',
        },
        'end': { # 종료 날짜
            'dateTime': '2024-04-19' + 'T10:00:00',
            'timeZone': 'Asia/Seoul',
        },
    }


In [ ]:
google_calendar.set_google_calendar(event)

##### 2. Open AI Api 연동해서 자동 예약 시스템 만들기

##### 2-1 사용자의 요청을 분류하는 에이전트 만들기

In [ ]:
from string import Template
prompt = Template("""
사용자의 요청은 다음과 같이 네 종류가 있습니다.

1. 오늘의 일정을 가져오기
2. 일정 id 리스트 가져오기
3. 일정 추가하기

사용자의 입력을 받아서 해당하는 요청과 관련된 번호를 반환해주세요.
번호 이외에 다른 것을 반환하면 안됩니다.
코드를 출력하면 안됩니다.

---
예시)
입력 : 성수동에서 4월 19일 9시부터 10시까지 회의 일정을 잡아줘.
출력 : 3

입력 : 오늘 처리해야 하는 일정을 알려줘.
출력 : 1

입력 : 일정 id 리스트를 알려줘.
출력 : 2

입력 : 4월 19일 9시부터 10시까지 회의 일정을 추가해줘.
출력 : 3

입력 : 일정 id 리스트 100개를 가져와줘.
출력 : 2

---

입력 : $input
출력 :
""")

print(prompt.substitute(input="오늘의 일정을 알려줘."))

In [ ]:
from openai import OpenAI

OPENAI_API_KEY= # OpenAI API key 입력
model_name="gpt-3.5-turbo"

client = OpenAI(
    api_key=OPENAI_API_KEY,
)

In [ ]:
system_prompt = """
너는 사용자의 요청을 분류해서 해당하는 기능을 수행하는 인공지능 챗봇이야
"""

def classify_intent(question):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": prompt.substitute(input=question)
            }
        ],
        model=model_name,
    )

    result = chat_completion.choices[0].message.content
    print(result)
    result = result[-1]
    try :
        result = int(result)
    except ValueError:
        print(f"ValueError : {result}")
        result = 0

    if result not in [1, 2, 3, 4]:
        print(f"Invalid result : {result}")
        return 0


    if result == 1:
        print("오늘의 일정을 가져오는 기능을 수행합니다.")
        return google_calendar.get_today_google_calendar()
    elif result == 2:
        print("일정 id 리스트를 가져오는 기능을 수행합니다.")
        return google_calendar.get_event_id_list()
    elif result == 3:
        print("일정 추가하기 기능을 수행합니다.")
        print("일정 추가에 필요한 정보를 포멧에 맞게 변환시켜주어야 합니다.")

result = classify_intent("오늘의 일정을 알려줘.")
print(result)

In [ ]:
result['items']

##### 2-2 사용자의 입력을 주어진 포맷에 맞춰서 바꿔서 일정 등록하기

In [ ]:
from string import Template

system_prompt_reservation = """
너는 사용자의 요청을 받아서 일정 변경 및 예약을 도와주는 인공지능 비서야.
주어진 포멧에 맞춰서 일정을 입력해줘.
"""

prompt_reservation = Template("""
사용자의 입력을 다음 포멧에 맞춰서 변경해줘.
아래 포맷 이외의 결과는 출력하면 안돼.


event = {
        'summary': '일정 제목',
        'location': '서울특별시',
        'description': '일정 설명',
        'start': {
            'dateTime': '2024-xx-xxTxx:xx:xx',
            'timeZone': 'Asia/Seoul',
        },
        'end': {
            'dateTime': '2024-xx-xxTxx:xx:xx',
            'timeZone': 'Asia/Seoul',
        },
    }

입력: $input
""")

## todo json mode
def make_reservation(event):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system_prompt_reservation
            },
            {
                "role": "user",
                "content": prompt_reservation.substitute(input=event)
            }
        ],
        model=model_name,
    )

    return chat_completion.choices[0].message.content

sample_event = """
성수동에서 4월 19일 9시부터 10시까지 회의 일정을 잡아줘.
"""

result = make_reservation(sample_event)
print(result)

In [ ]:
def post_processing_reservation_result(result):
    feature_name_list = ["summary", "location", "description"]
    feature_time_list = ["start", "end"]

    results = {}
    for feature in feature_name_list:
        feature_name_index = result.find(feature)
        feature_value_start_index = feature_name_index + len(feature) + 4

        feature_value_end_index = result.find("'", feature_value_start_index)
        feature_value = result[feature_value_start_index:feature_value_end_index]
        print(f"{feature} : {feature_value}")
        results[feature] = feature_value

    for feature in feature_time_list:
        feature_name_index = result.find(feature)
        feature_value_start_index = result.find("dateTime", feature_name_index) + len("dateTime") + 4

        feature_value_end_index = result.find("'", feature_value_start_index)
        feature_value = result[feature_value_start_index:feature_value_end_index]

        time_zone_start_index = result.find("timeZone", feature_name_index) + len("timeZone") + 4
        time_zone_end_index = result.find("'", time_zone_start_index)
        time_zone = result[time_zone_start_index:time_zone_end_index]

        print(f"{feature} : {feature_value}")
        print(f"timeZone : {time_zone}")

        results[feature] = {"dateTime": feature_value, "timeZone": time_zone}


    return results
post_processing_reservation_result(result)

In [ ]:
google_calendar.set_google_calendar(post_processing_reservation_result(result))

##### 2-3 일정 예약 기능 추가하기

In [ ]:
system_prompt = """
너는 사용자의 요청을 분류해서 해당하는 기능을 수행하는 인공지능 챗봇이야
"""

def classify_intent(question):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": prompt.substitute(input=question)
            }
        ],
        model=model_name,
    )
    print(prompt.substitute(input=question))
    result = chat_completion.choices[0].message.content
    print(result)
    result = result[-1]
    try :
        result = int(result)
    except ValueError:
        print(f"ValueError : {result}")
        result = 0

    if result not in [1, 2, 3, 4]:
        print(f"Invalid result : {result}")
        return 0


    if result == 1:
        print("오늘의 일정을 가져오는 기능을 수행합니다.")
        return google_calendar.get_today_google_calendar()
    elif result == 2:
        print("일정 id 리스트를 가져오는 기능을 수행합니다.")
        return google_calendar.get_event_id_list()
    elif result == 3:
        print("일정 추가하기 기능을 수행합니다.")
        result = make_reservation(question)
        result = post_processing_reservation_result(result)
        google_calendar.set_google_calendar(result)

print(classify_intent("오늘의 일정을 알려줘."))

In [ ]:
print(prompt.substitute(input="장소를 서울특별시로 하고, 5월 6일 오후 9시부터 10시까지 회의 일정을 추가해줘."))

In [ ]:
classify_intent("회의 일정을 잡아줘. 장소는 서울특별시 역삼동이고, 5월 6일 오후 9시부터 10시까지 진행될거야.")

**reference**

[https://velog.io/@kho5420/Python-구글-Open-API로-캘린더-활용해보기](https://velog.io/@kho5420/Python-%EA%B5%AC%EA%B8%80-Open-API%EB%A1%9C-%EC%BA%98%EB%A6%B0%EB%8D%94-%ED%99%9C%EC%9A%A9%ED%95%B4%EB%B3%B4%EA%B8%B0)

[https://velog.io/@kho5420/Python-구글-Open-API로-캘린더-활용해보기-2-일정추가](https://velog.io/@kho5420/Python-%EA%B5%AC%EA%B8%80-Open-API%EB%A1%9C-%EC%BA%98%EB%A6%B0%EB%8D%94-%ED%99%9C%EC%9A%A9%ED%95%B4%EB%B3%B4%EA%B8%B0-2-%EC%9D%BC%EC%A0%95%EC%B6%94%EA%B0%80)